# Let's Code - Criação de um pipeline dentro do Sklearn:


- O que é um pipeline?
- Quais as vantagens de se utilizar um pipeline?
- E o pipeline dentro do contexto de ciência de dados?
    - Feature normalization
    - Feature selection
    - Treinando e avaliando modelos em Pipelines
        - Cross Validation em Pipelines
        - Grid-Search com Pipelines
    - Visualizando o pipeline
- O pipeline em aplicação no dia-a-dia do cientista de dados



## O que é um pipeline?


Um pipeline (em tradução livre uma sequência de canos conectados) é uma sequência de funções conectadas que ao final nos dá um resultado esperado.

Considere o exemplo abaixo onde várias funções estão sendo aplicadas sob uma string:

#### Exemplo 1: Implementação sem pipes

In [1]:
import numpy as np

In [2]:
string = "O rato roeu a roupa do rei de roma"

# deixa tudo como letra minuscula
string = string.lower()

# substitui a palavra rato por rata
string = string.replace("rato", "rata")

# substitui rei por rainha
string = string.replace("rei", "rainha")

# substitui o por a
string = string.replace("o ", "a ")

# substitui "do" por "da"
string = string.replace("do", "da")

# deixa a primeira letra maiúscula
string = string.capitalize()
print(string)

A rata roeu a roupa da rainha de roma


#### Como podemos melhorar o código acima?

In [3]:
string = "O rato roeu a roupa do rei de roma"

string = string.lower()

replace = {"rato": "rata", "rei": "rainha", "o ": "a ", "do": "da"}

for original, replacer in replace.items():
    
    string = string.replace(original, replacer)

string = string.capitalize()

#### Exemplo 3: Utilizando funções customizadas

In [5]:
def to_str(value):
    
    return str(value) + " é a resposta para tudo"

In [6]:
string = to_str(42)

# todas as palavras com a primeira letra maiuscula
string = string.title()

# RESPOSTA agora é toda maiúscula
string = string.replace("Resposta", "RESPOSTA")

#### Exemplo 4: Pipeline com funções customizadas

In [7]:
to_str(42).title().replace("Resposta", "RESPOSTA")

'42 É A RESPOSTA Para Tudo'

## Quais as vantagens de se utilizar um pipeline?

Os códigos acima são exemplos básicos da utilização de um pipeline. Dentro de códigos pequenos talvez não seja possível ver tanta a necessidade da utilização de pipes em funções, entretanto, em grandes projetos que exigem muitas linhas de código de dedicação esse tipo de técnica se torna indispensável. <br>

Assim as principais vantagens de se utilizar um pipeline são:

- Aumento na legibilidade de código
- Aumento na reproductibilidade dos resultados


Entretanto, vale ressaltar, que o uso excessivo de pipes pode atrapalhar um pouco na legibilidade. Vai do bom senso do profissional ver até que ponto o uso trás benefícios pra legibilidade do código.

## E o pipeline dentro do contexto de ciência de dados?

O pipeline em aprendizado de máquina foi introduzido pela biblioteca Sklearn com o objetivo de aumentar a legibilidade de código evitando a reutilização excessiva de variáveis. Outro ponto é a reproductibilidade dos experimentos e sistemas que são feitos com base em modelos do Sklearn

<br>

Para colocar tudo isso que nós vimos em prática, vamos passar por um stack completo de operações utilizando o pipeline. As operações que serão realizadas são:

- Feature Normalization
- Feature Selection
- Grid Search
- Cross-Validation

Para este exemplo vamos utilizar os dados do dataset heart disease: <b><a href="https://www.kaggle.com/ronitf/heart-disease-uci">Heart Disease</a></b>. O objetivo neste dataset é predizer se um paciente possui uma doença cardíaca ou não.

In [8]:
import pandas as pd

import numpy as np

In [9]:
df = pd.read_table("Data/heart.csv", sep=',')

In [10]:
df

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,M,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,M,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,F,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,M,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,F,0,120,354,0,1,163,1,0.6,2,0,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,57,F,0,140,241,0,1,123,1,0.2,1,0,3,0
299,45,M,3,110,264,0,1,132,0,1.2,1,0,3,0
300,68,M,0,144,193,1,1,141,0,3.4,1,2,3,0
301,57,M,0,130,131,0,1,115,1,1.2,1,1,3,0


In [11]:
df.describe()

,age,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
count,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000
mean,54.366337,0.966997,131.623762,246.264026,0.148515,0.528053,149.646865,0.326733,1.039604,1.399340,0.729373,2.313531,0.544554
std,9.082101,1.032052,17.538143,51.830751,0.356198,0.525860,22.905161,0.469794,1.161075,0.616226,1.022606,0.612277,0.498835
min,29.000000,0.000000,94.000000,126.000000,0.000000,0.000000,71.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,47.500000,0.000000,120.000000,211.000000,0.000000,0.000000,133.500000,0.000000,0.000000,1.000000,0.000000,2.000000,0.000000
50%,55.000000,1.000000,130.000000,240.000000,0.000000,1.000000,153.000000,0.000000,0.800000,1.000000,0.000000,2.000000,1.000000
75%,61.000000,2.000000,140.000000,274.500000,0.000000,1.000000,166.000000,1.000000,1.600000,2.000000,1.000000,3.000000,1.000000
max,77.000000,3.000000,200.000000,564.000000,1.000000,2.000000,202.000000,1.000000,6.200000,2.000000,4.000000,3.000000,1.000000


Como podemos ver na tabela acima, temos como grande maioria <b>variáveis númericas contínuas</b>, a variável target é o que queremos predizer e a variável <b>sex</b> é uma variável categórica no tipo <b>string</b>.

<br>

<b>Dado que valores muito discrepantes podem ter um impacto muito grande no modelo iremos:</b>

- Normalizar as variáveis contínuas

<b>Dado que modelos do Sklearn só aceitam variáveis númericas temos que:</b>

- Categorizar a variável do tipo string para valor númerico

<b>Dado que nem todas as features auxiliam na predição do modelo:</b>

- Seleção das K melhores features

<b>Dado que temos que garantir a validade estatística dos nossos algoritmos:</b>

- K-Fold

<b>Dado que temos que queremos ter o melhor algoritmo possível:</b>

- GridSearch

Resumo de algumas funções que utilizaremos

<img src="Figures/cross_validation.png">
<center><b>Relembrando: Validação Cruzada</b></center>


<img src="Figures/f1.jpeg">
<center><b>Relembrando: F1-Score</b></center>


<img src="Figures/Precisão_e_revocação.png" width=400 heigth=400>
<center><b>Relembrando: F1 Score é a combinação das métricas de precisão e revocação</b></center>

In [12]:
from sklearn.pipeline import Pipeline

<b> Primeiramente vamos importar a função de <b>Pipeline</b> do sklearn.</b>

In [12]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer
from sklearn.metrics import f1_score
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, mutual_info_classif

In [13]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer
from sklearn.metrics import f1_score
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, mutual_info_classif

In [14]:
df_aux = df.copy()

imputer = SimpleImputer(strategy='most_frequent')

all_columns = list(df_aux.drop(['target'], axis=1).columns)

df_aux[all_columns] = imputer.fit_transform(df_aux[all_columns])

encoding = LabelEncoder()

df_aux['sex'] = encoding.fit_transform(df_aux['sex'])

# z = (x - u) / s
# Remove a variância da média
normalization = StandardScaler()

df_aux[all_columns] = normalization.fit_transform(df_aux.drop(['target'], axis=1))

model = RandomForestClassifier()

# # definindo 5 folds em que os dados vão ser divididos
kfold = KFold(n_splits=5, shuffle=True)

# # Realiza a valiação cruzada dos resultados e 
# # nos dá como retorno os resultados de cada um dos folds
scoring = make_scorer(f1_score)

results = cross_validate(model, X=df_aux.drop(['target'], axis=1), y=df_aux['target'], cv=kfold, scoring=scoring)

<b>Sem pipeline</b>

In [15]:
df_aux = df.copy()

imputer = SimpleImputer(strategy='most_frequent')

all_columns = list(df_aux.drop(['target'], axis=1).columns)

df_aux[all_columns] = imputer.fit_transform(df_aux[all_columns])

encoding = LabelEncoder()

df_aux['sex'] = encoding.fit_transform(df_aux['sex'])

# z = (x - u) / s
# Remove a variância da média
normalization = StandardScaler()

df_aux[all_columns] = normalization.fit_transform(df_aux.drop(['target'], axis=1))

model = RandomForestClassifier()

# # definindo 5 folds em que os dados vão ser divididos
kfold = KFold(n_splits=5, shuffle=True)

# # Realiza a valiação cruzada dos resultados e 
# # nos dá como retorno os resultados de cada um dos folds
scoring = make_scorer(f1_score)

results = cross_validate(model, X=df_aux.drop(['target'], axis=1), y=df_aux['target'], cv=kfold, scoring=scoring)

In [17]:
# criando o pipeline

# cada passo é declarado como um dupla
# o primeiro valor descreve qual operação será aplicada
# o segundo valor possui a função que vai ser aplicada sobre os dados
# vale ressaltar que o segundo valor pode ser uma função personalizada
# desde que o modelo possua as funções fit, transform


df_aux = df.drop(['sex'], axis=1)

model = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('normalization', StandardScaler()),
    ('model', RandomForestClassifier())
])


# definindo 5 folds em que os dados vão ser divididos
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

# Realiza a valiação cruzada dos resultados e 
# nos dá como retorno os resultados de cada um dos folds
sg = make_scorer(f1_score)

results = cross_validate(model, X=df_aux.drop(['target'], axis=1), y=df_aux['target'], cv=kfold, scoring=sg)

<b>Modelo simples sem a variável sexo</b>

<b>Modelo simples com a variável sexo</b>

In [18]:
imputer = Pipeline(steps=[('imputer', SimpleImputer(strategy='most_frequent'))])

encoding = Pipeline(steps=[('encoding', OneHotEncoder())])

normalization = Pipeline(steps=[('normalization', StandardScaler())])

pre_processing = ColumnTransformer(transformers=[
                                                    ('cat', encoding, ['sex']),
                                                    ('normalization', normalization, list(df.drop(['sex', 'target'], axis=1).columns)),
                                                    ('imputer', imputer, list(df.drop(['sex', 'target'], axis=1).columns))
                                                ])

model = Pipeline(steps=[
    ('pre processing', pre_processing),
    ('model', RandomForestClassifier())
])


# definindo 5 folds em que os dados vão ser divididos
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

# Realiza a valiação cruzada dos resultados e 
# nos dá como retorno os resultados de cada um dos folds
scoring = make_scorer(f1_score)

results = cross_validate(model, X=df.drop(['target'], axis=1), y=df['target'], cv=kfold, scoring=scoring)

<b>Modelo simples com a variável sexo</b>

In [19]:
imputer = Pipeline(steps=[('imputer', SimpleImputer(strategy='most_frequent'))])

encoding = Pipeline(steps=[('encoding', OneHotEncoder())])

normalization = Pipeline(steps=[('normalization', StandardScaler())])

pre_processing = ColumnTransformer(transformers=[
                                                    ('cat', encoding, ['sex']),
                                                    ('normalization', normalization, list(df.drop(['sex', 'target'], axis=1).columns)),
                                                    ('imputer', imputer, list(df.drop(['sex', 'target'], axis=1).columns))
                                                ])

model = Pipeline(steps=[
    ('pre processing', pre_processing),
    ('model', RandomForestClassifier())
])


# definindo 5 folds em que os dados vão ser divididos
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

# Realiza a valiação cruzada dos resultados e 
# nos dá como retorno os resultados de cada um dos folds
scoring = make_scorer(f1_score)

results = cross_validate(model, X=df.drop(['target'], axis=1), y=df['target'], cv=kfold, scoring=scoring)

In [20]:
"F1-Score - Mean: ", np.mean(results['test_score']), " Std. Dev ", np.std(results['test_score'])

('F1-Score - Mean: ', 0.8277704259042729, ' Std. Dev ', 0.03140316907378237)

In [21]:
from sklearn.model_selection import GridSearchCV

### Grid Search com pipelines

In [22]:
model = Pipeline(steps=[
    ('pre processing', pre_processing),
    ('feature-selection', SelectKBest(mutual_info_classif, k = 10)),
    ('model', RandomForestClassifier())
])

In [23]:
# Parâmetros que serão explorados
parameters = {'model__max_depth': [3, 4, 5], 'model__criterion': ['gini', 'entropy'],
              'model__n_estimators': [50, 100, 200]}



kfold = KFold(n_splits=5, shuffle=True, random_state=42)

# Identifica os melhores parâmetros
grid = GridSearchCV(model, param_grid=parameters, cv=kfold, n_jobs=-1)


In [22]:
from sklearn import set_config

In [26]:
result = grid.fit(X=df.drop(['target'], axis=1), y=df['target'])

# os melhores parâmetros
print(grid.cv_results_)

{'mean_fit_time': array([0.61168041, 0.85586705, 1.43369036, 0.62425489, 0.91315827,
       1.42289705, 0.65052152, 0.81536303, 1.59029441, 0.54441118,
       0.83891263, 1.41301098, 0.54306092, 0.85080886, 1.39686551,
       0.63139997, 0.94073043, 1.35391431]), 'std_fit_time': array([0.15026489, 0.15832707, 0.16792075, 0.12370889, 0.08767346,
       0.1561126 , 0.10202413, 0.15419864, 0.30631617, 0.10832436,
       0.1963884 , 0.22336445, 0.05920661, 0.19830678, 0.15548187,
       0.11786565, 0.0931329 , 0.36185592]), 'mean_score_time': array([0.05051942, 0.06687574, 0.1254426 , 0.05601273, 0.07593894,
       0.10215774, 0.05595064, 0.0757153 , 0.12029252, 0.04807901,
       0.06488028, 0.11164613, 0.0446424 , 0.06434999, 0.11589646,
       0.05161915, 0.06606574, 0.08182831]), 'std_score_time': array([0.01326989, 0.01536339, 0.03418682, 0.01963976, 0.0120756 ,
       0.03120902, 0.0108959 , 0.01800043, 0.01283599, 0.01354053,
       0.02252115, 0.03169701, 0.013537  , 0.02068995, 0.

In [23]:
set_config(display='diagram')

In [24]:
from sklearn.utils import estimator_html_repr
from IPython.display import HTML

In [25]:
with open('my_estimator.html', 'w') as f:

    f.write(estimator_html_repr(model))

In [26]:
HTML('my_estimator.html')